In [280]:
def create_ngrams(tokens,n):
    sequences = [tokens[i:] for i in range(n)]
    ngrams = [' '.join(ngram) for ngram in list(zip(*sequences))]
    return ngrams, list(set(ngrams))


def create_word_mapping(values_list):
        values_list.append('<| unknown |>')
        value2ind = {value: ind for ind, value in enumerate(values_list)}
        ind2value = dict(enumerate(values_list))
        return value2ind, ind2value

def create_transition_prob_matrix(input_text,n):
    
    # n is for selecting the ngram
    import re

    punctuation_pad = '!?.,:-;'
    punctuation_remove = '"()_\n'


    content_preprocess = re.sub(r'(\S)(\n)(\S)', r'\1 \2 \3', input_text)
    # Removing charecter like \n\n
    content_preprocess = content_preprocess.translate(str.maketrans('', '', punctuation_remove))
    # Considering , . as different charecter to be used as a state in the transition probability matri
    content_preprocess = content_preprocess.translate(str.maketrans({key: ' {0} '.format(key) for key in punctuation_pad}))
    # Removing extra space between words or charecters
    content_preprocess = re.sub(' +', ' ', content_preprocess)
    # remove leading and trailing whitespaces
    content = content_preprocess.strip()
    
    tokens = content.split(' ')
    token_dist = list(set(tokens))
    

    
    ngram, ngram_dist = create_ngrams(tokens,n)
    
    token2ind, ind2token = create_word_mapping(token_dist) # distinct token
    ngram2ind, ind2ngram = create_word_mapping(ngram_dist) # distinct ngram
    
    #tokens_ind = [token2ind[token] if token in token2ind.keys() else token2ind['<| unknown |>']
     #                      for token in tokens]

    
    
    row_ind, col_ind, values = [], [], []

    for i in range(len(tokens[:-n])):
        ngram = ' '.join(tokens[i:i + n])
        ngram_ind = ngram2ind[ngram]
        next_word_ind = token2ind[tokens[i + n]]

        row_ind.extend([ngram_ind])
        col_ind.extend([next_word_ind])
        values.extend([1])

    S = scipy.sparse.coo_matrix((values, (row_ind, col_ind)), shape=(len(ngram2ind), len(token2ind)))
    
    P = normalize(S, norm='l1', axis=1)
    return P



    
def powr(P,order):
    import numpy as np
    k = len(P)
    M = np.identity(k)
    for i in range(order):
        M = np.dot(M,P)
    return M


def transition_probability_matrix(input_text,n,order):
    if order == 1:
        P = create_transition_prob_matrix(input_text,n)
    else:
        P = create_transition_prob_matrix(input_text,n)
        P = P.toarray()
        P = powr(P,order)
        P = scipy.sparse.coo_matrix(P)
        P = P.tocsr()
    return P

# Loading the train data

In [220]:
path = 'E:/NMT MS/Spring 22/MATH 486/Project/game_of_thrones/game_of_thrones.txt'
input_text = open(path, 'r', encoding='utf-8').read()

In [221]:
A = create_transition_prob_matrix(input_text,n=2)

In [222]:
A.shape

(115200, 13951)

In [223]:
print(A)

  (0, 13921)	1.0
  (1, 6689)	1.0
  (2, 9456)	1.0
  (3, 3461)	0.14285714285714285
  (3, 4542)	0.2857142857142857
  (3, 4867)	0.14285714285714285
  (3, 5648)	0.14285714285714285
  (3, 10308)	0.14285714285714285
  (3, 13921)	0.14285714285714285
  (4, 7052)	1.0
  (5, 4290)	1.0
  (6, 6248)	1.0
  (7, 10768)	1.0
  (8, 1074)	0.5
  (8, 5490)	0.5
  (9, 4015)	1.0
  (10, 12956)	1.0
  (11, 2547)	0.25
  (11, 7806)	0.25
  (11, 9163)	0.25
  (11, 12724)	0.25
  (12, 12363)	1.0
  (13, 4290)	1.0
  (14, 4290)	1.0
  (15, 6573)	1.0
  :	:
  (115180, 9114)	1.0
  (115181, 12964)	1.0
  (115182, 1611)	0.3333333333333333
  (115182, 6421)	0.3333333333333333
  (115182, 9223)	0.3333333333333333
  (115183, 12405)	1.0
  (115184, 12378)	1.0
  (115185, 12114)	1.0
  (115186, 810)	1.0
  (115187, 4290)	1.0
  (115188, 5665)	1.0
  (115189, 1611)	0.6666666666666666
  (115189, 13189)	0.3333333333333333
  (115190, 3463)	0.25
  (115190, 12974)	0.25
  (115190, 13189)	0.25
  (115190, 13690)	0.25
  (115191, 13189)	1.0
  (115192, 429

In [213]:
AA = A.toarray()

In [216]:
AAA = powr(AA,n=10)

In [217]:
print(scipy.sparse.coo_matrix(AAA))

  (0, 0)	5.800547028423785e-06
  (0, 1)	2.892887600369397e-06
  (0, 2)	2.912172250746355e-06
  (0, 3)	2.893310467910931e-06
  (0, 4)	8.736532207123111e-06
  (0, 5)	2.890071509263789e-06
  (0, 6)	5.5181241023822915e-05
  (0, 7)	1.157575649401192e-05
  (0, 8)	2.0351139118643596e-05
  (0, 9)	5.80613786618147e-06
  (0, 10)	2.9046754841551878e-06
  (0, 11)	0.00022074075064986302
  (0, 12)	5.820579645763569e-06
  (0, 13)	2.9035273450233724e-06
  (0, 14)	2.9005093829926192e-06
  (0, 15)	5.803789002168645e-06
  (0, 16)	5.865883825239653e-06
  (0, 17)	2.9075245750175298e-06
  (0, 18)	2.9113156660688543e-06
  (0, 19)	8.149556753622806e-05
  (0, 20)	2.9234815536531974e-06
  (0, 21)	2.9050668489093347e-06
  (0, 22)	2.892072758735196e-06
  (0, 23)	1.1601086130865655e-05
  (0, 24)	2.889527321445664e-06
  :	:
  (13949, 13925)	0.0007204370638158197
  (13949, 13926)	8.412959962844215e-05
  (13949, 13927)	2.9003776477341782e-06
  (13949, 13928)	1.4512472552583043e-05
  (13949, 13929)	2.8965773873283516e

# Generate sentance

In [281]:
def pre_process(input_text):
    # n is for selecting the ngram
    import re

    punctuation_pad = '!?.,:-;'
    punctuation_remove = '"()_\n'


    content_preprocess = re.sub(r'(\S)(\n)(\S)', r'\1 \2 \3', input_text)
    # Removing charecter like \n\n
    content_preprocess = content_preprocess.translate(str.maketrans('', '', punctuation_remove))
    # Considering , . as different charecter to be used as a state in the transition probability matri
    content_preprocess = content_preprocess.translate(str.maketrans({key: ' {0} '.format(key) for key in punctuation_pad}))
    # Removing extra space between words or charecters
    content_preprocess = re.sub(' +', ' ', content_preprocess)
    # remove leading and trailing whitespaces
    content = content_preprocess.strip()
    return content
    


def get_ngram(input_text,n):

    content = pre_process(input_text)    
    tokens = content.split(' ')
    token_dist = list(set(tokens))
    
    sequences = [tokens[i:] for i in range(n)]
    ngrams = [' '.join(ngram) for ngram in list(zip(*sequences))]
    
    
    #ngram, ngram_dist = create_ngrams(tokens,n)
    
    return ngrams

def check_prefix(input_text,prefix,n):
    prefix_list = prefix.split(' ')[-n:]
    ngrams = get_ngram(input_text,n)
    if len(prefix_list) < n:
        warnings.warn(
                'Prefix is too short, please provide prefix of length: %d. Random ngram used instead.' % n)
        return np.random.choice(ngrams)
    else:
        prefix = ' '.join(prefix_list)
        if prefix in ngrams:
            return prefix
        else:
            warnings.warn(
                    'Prefix is not included in ngrams of the model. Provide another prefix. Random ngram used instead.')
            return np.random.choice(ngrams)

        
def return_next_word(input_text,prefix,n,order):
    prefix = check_prefix(input_text,prefix,n)
    content = pre_process(input_text)
 
    
    tokens = content.split(' ')
    token_dist = list(set(tokens))
    

    
    ngram, ngram_dist = create_ngrams(tokens,n)
    token2ind, ind2token = create_word_mapping(token_dist) # distinct token
    ngram2ind, ind2ngram = create_word_mapping(ngram_dist) # distinct ngram
    
    # Transition prob matrix
    #transition_matrix_prob = create_transition_prob_matrix(input_text,n)
    transition_matrix_prob = transition_probability_matrix(input_text,n,order)
    prefix_ind = ngram2ind[prefix]
    weights = transition_matrix_prob[prefix_ind].toarray()[0]
    token_ind = np.random.choice(range(len(weights)), p=weights)
    next_word = ind2token[token_ind]
    return next_word

def reverse_preprocess(text):
    text_reverse = re.sub(r'\s+([!?"\'().,;-])', r'\1', text)
    text_reverse = re.sub(' +', ' ', text_reverse)
    return text_reverse



def generate_sequence(input_text,prefix,n, k,order):
    prefix = check_prefix(input_text,prefix,n)
    sequence = prefix.split(' ')

    for i in range(k):
        next_word = return_next_word(input_text,prefix,n,order)
        sequence.append(next_word)
        prefix = ' '.join(sequence[-n:])

    return reverse_preprocess(' '.join(sequence))

In [278]:
generate_sequence(input_text,prefix = "Snow",n = 1,k=10,order=1)

'Snow. And her silver mockingbird cape and long trestle table'

In [ ]:
generate_sequence(input_text,prefix = "Snow",n = 1,k=10,order=3)